In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/workspaces/SebastianBot")

In [2]:
from cloud.functions.infrastructure.google.helper import load_google_credentials


credentials = load_google_credentials()

In [3]:
from googleapiclient.discovery import build

service = build(
"drive", "v3", credentials=credentials, cache_discovery=False
)

Setup:

In [4]:
from io import BytesIO


testi_file_content = b"Hello, this is a test file."
bytes_content = BytesIO(testi_file_content)

This is how to upload a file to google drive

In [5]:
from googleapiclient.http import MediaIoBaseUpload

filename = "example.txt"
drive_folder_id = "1VGX5Wt8D3huZm3vVemjI3C6zz6W38PJr"
mime_type = "application/octet-stream"

file_metadata = {"name": filename, "parents": [drive_folder_id]}

media = MediaIoBaseUpload(bytes_content, mimetype=mime_type, resumable=True)

uploaded_file = (
service.files()  # type: ignore
.create(
    body=file_metadata,
    media_body=media,
    fields="id",  # Only return the file ID
)
.execute()
)

# Using GoogleDriveClient

In [6]:
from cloud.dependencies.clients import resolve_google_drive_client


client = resolve_google_drive_client()

In [7]:
from sebastian.clients.google.drive.models import UploadFileRequest


testi_file_content = b"Hello, this is another test file."
request = UploadFileRequest(
    filename="example2.txt",
    content=testi_file_content,
    folder_id="1VGX5Wt8D3huZm3vVemjI3C6zz6W38PJr"
)

client.upload_file(request)

UploadFileResponse(file_id='1rDmiNFmPcYv2cLzcIhSh6xs79OP421lt')

# Uploading with path

If the filename in UploadFileRequest contains a path (i.e. filename="sub1/sub2/example.txt"), upload_file should create those folders if they don't exist yet

## Step 1: Find or create folder by name within a parent folder

In [8]:
def find_or_create_folder(service, folder_name: str, parent_id: str) -> str:
    """Find a folder by name within a parent folder, or create it if it doesn't exist.
    
    Args:
        service: Google Drive API service instance
        folder_name: Name of the folder to find or create
        parent_id: ID of the parent folder
        
    Returns:
        The ID of the found or created folder
    """
    # Search for existing folder
    query = f"name='{folder_name}' and '{parent_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    items = results.get("files", [])
    
    if items:
        # Folder exists, return its ID
        return items[0]["id"]
    
    # Folder doesn't exist, create it
    file_metadata = {
        "name": folder_name,
        "mimeType": "application/vnd.google-apps.folder",
        "parents": [parent_id]
    }
    folder = service.files().create(body=file_metadata, fields="id").execute()
    return folder["id"]

Test finding/creating a single folder:

In [9]:
base_folder_id = "1VGX5Wt8D3huZm3vVemjI3C6zz6W38PJr"
folder_id = find_or_create_folder(service, "test_folder", base_folder_id)
print(f"Folder ID: {folder_id}")

Folder ID: 1-0sVlB-qEecW_ZQpohQxnuX_3uE6PBCc


## Step 2: Navigate/create nested folder path

In [10]:
def ensure_folder_path(service, path: str, base_folder_id: str) -> str:
    """Navigate or create a folder path, returning the ID of the final folder.
    
    Args:
        service: Google Drive API service instance
        path: Folder path like "sub1/sub2/sub3"
        base_folder_id: ID of the base folder to start from
        
    Returns:
        The ID of the final folder in the path
    """
    current_parent_id = base_folder_id
    
    # Split path and create each folder in sequence
    for folder_name in path.split("/"):
        if folder_name:  # Skip empty strings
            current_parent_id = find_or_create_folder(service, folder_name, current_parent_id)
    
    return current_parent_id

Test creating nested folder path:

In [11]:
base_folder_id = "1VGX5Wt8D3huZm3vVemjI3C6zz6W38PJr"
nested_path = "sub1/sub2"
final_folder_id = ensure_folder_path(service, nested_path, base_folder_id)
print(f"Final folder ID for '{nested_path}': {final_folder_id}")

Final folder ID for 'sub1/sub2': 1Dqw8bODmx8FlmpfqM-iLnOup11HzSTHE


## Step 3: Upload file with nested path

In [12]:
import os

def upload_file_with_path(service, filename_with_path: str, content: bytes, base_folder_id: str, mime_type: str = "application/octet-stream") -> str:
    """Upload a file to Google Drive, creating nested folders as needed.
    
    Args:
        service: Google Drive API service instance
        filename_with_path: Path like "sub1/sub2/example.txt"
        content: File content as bytes
        base_folder_id: ID of the base folder
        mime_type: MIME type of the file
        
    Returns:
        The ID of the uploaded file
    """
    # Split the path into directory and filename
    dir_path = os.path.dirname(filename_with_path)
    filename = os.path.basename(filename_with_path)
    
    # Navigate/create the folder structure
    if dir_path:
        parent_folder_id = ensure_folder_path(service, dir_path, base_folder_id)
    else:
        parent_folder_id = base_folder_id
    
    # Upload the file to the final folder
    bytes_content = BytesIO(content)
    file_metadata = {"name": filename, "parents": [parent_folder_id]}
    media = MediaIoBaseUpload(bytes_content, mimetype=mime_type, resumable=True)
    
    uploaded_file = (
        service.files()
        .create(body=file_metadata, media_body=media, fields="id")
        .execute()
    )
    
    return uploaded_file["id"]

Test uploading a file with nested path:

In [13]:
base_folder_id = "1VGX5Wt8D3huZm3vVemjI3C6zz6W38PJr"
test_content = b"Hello, this is a file uploaded with path!"
file_path = "sub1/sub2/example.txt"

file_id = upload_file_with_path(service, file_path, test_content, base_folder_id)
print(f"File uploaded successfully! File ID: {file_id}")

File uploaded successfully! File ID: 1QEKhN6LLeKppY7fg7XWxfDMtah58Rq5a


## Testing updated GoogleDriveClient with nested paths

In [14]:
# Reload the client to get the updated code
from cloud.dependencies.clients import resolve_google_drive_client
from sebastian.clients.google.drive.models import UploadFileRequest

client = resolve_google_drive_client()

# Test with nested path
test_content = b"Testing nested path with updated client!"
request = UploadFileRequest(
    filename="nested_test/subfolder/test_file.txt",
    content=test_content,
    folder_id="1VGX5Wt8D3huZm3vVemjI3C6zz6W38PJr"
)

result = client.upload_file(request)
print(f"File uploaded successfully! File ID: {result.file_id}")

File uploaded successfully! File ID: 1b6Ua7DaKSX5UqnUqjRYJIOyLJ-E2WJJM
